# Brain Spark : Octomap

## 1. Space of map saved on disk comparison

In [1]:
import numpy as np
import open3d as o3d
import os
from scripts.regular_voxel_grid import RegularVoxelGrid, visualize_voxel_grid

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
voxel_grid = RegularVoxelGrid(resolution=0.25, max_coord=10, min_coord=-10)

In [3]:
voxel_grid.create_regular_voxel_map()
visualize_voxel_grid(voxel_grid.get_voxel_grid_points())

[Open3D WARNING] GLFW Error: X11: Failed to open display :1
[Open3D WARNING] Failed to initialize GLFW
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.


Authorization required, but no authorization protocol specified


In [4]:
voxel_grid.save_voxel_grid_ply()
print(f"Saved voxel map representation space : {voxel_grid.get_saved_voxel_map_disk_size()} MB")

Saved voxel map representation space : 0.5860786437988281 MB


In [5]:
voxel_grid.write_voxel_grid_points_to_txt()


### Creating the octomap in CPP and saving it to disk

In [6]:
# Create an Open3D Octree for visualization first
octree = o3d.geometry.Octree(max_depth=8)  # The tree is stopped when the depth level reaches 8
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(voxel_grid.get_voxel_grid_points())
octree.convert_from_point_cloud(pcd, size_expand=0.02)

# Visualize the Octree
o3d.visualization.draw_geometries([octree])

[Open3D WARNING] GLFW Error: X11: Failed to open display :1
[Open3D WARNING] Failed to initialize GLFW
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.


Authorization required, but no authorization protocol specified


Since open3d does not allow us to save the octree, I am going to use the official Octomap repository to create the same octomap using the above points, and then save it to a usable binary file. And then check the saved file size. 

In [8]:
!cp simple_example.cpp octomap/octomap/src
if not os.path.exists('octomap/build'):
    os.makedirs('octomap/build', exist_ok=True)
!cd octomap/build && cmake .. && make
!./octomap/bin/simple_example  # will create the octomap corresponding to above voxel grid points and save it to a binary file

-- 

-- octomap building as Release
-- Could NOT find Doxygen (missing: DOXYGEN_EXECUTABLE) 
-- 

-- Compile octomap using: make
-- Install octomap using: make install
--     (be sure to set the correct CMAKE_INSTALL_PREFIX before)
-- Compile API-documentation using: make docs

-- 

-- octovis building as Release
-- Found octomap version: 1.10.0
-- Could NOT find OpenGL (missing: OPENGL_opengl_LIBRARY OPENGL_glx_LIBRARY OPENGL_INCLUDE_DIR) 
-- QGLViewer includes found in /home/brain-spark/Octomap/octomap/octovis/src/extern/QGLViewer
-- QGLViewer library not found.
-- Trying to build libQGLViewer from source in /home/brain-spark/Octomap/octomap/octovis/src/extern/QGLViewer
-- 	 generating Makefile using qmake
-- 	 building library (this may take some time...)
make: *** No targets specified and no makefile found.  Stop.
CMake Warning at octovis/CMakeModules/FindQGLViewer.cmake:111 (MESSAGE):
  Could not find libQGLViewer.so, failed to build?
Call Stack (most recent call first):
  octovis

In [10]:
print(f"Saved octomap space: {os.path.getsize('octree_from_point_cloud.bt') / (1024 * 1024)} MB")

Saved octomap space: 0.06645870208740234 MB


In [11]:
# Visualize the octomap
!octovis octree_from_point_cloud.bt

Authorization required, but no authorization protocol specified
qt.qpa.xcb: could not connect to display :1
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: eglfs, linuxfb, minimal, minimalegl, offscreen, vnc, xcb.

Aborted (core dumped)


## 2. Dynamically adding points comparison

In [12]:
### Another advantage of using octomap is that we can dynamically add points to the octomap and update it, no matter if the current point being added is very far from the existing points. In a regular voxel grid, if we want to add a point that is out of bounds from the initialized voxel grid space, we need to reinitialize the voxel grid with a larger space. This would require us to reinitialize the voxel grid and save it to disk again.

In [13]:
# Add a point to the voxel grid
voxel_grid.add_point_to_voxel_grid(np.array([20, -6, 0]))  # This point is out of bounds of the current voxel grid space. Hence trying to add will result in an error

IndexError: index 120 is out of bounds for axis 0 with size 80

### Adding a point to the octomap 

#### Modifying simple_example.cpp to add a point to the octomap that is far away from existing points 

In [15]:
!cp simple_example.cpp octomap/octomap/src
!cd octomap/build && cmake .. && make
!./octomap/bin/simple_example
!octovis octree_from_point_cloud.bt

-- 

-- octomap building as Release
-- Could NOT find Doxygen (missing: DOXYGEN_EXECUTABLE) 
-- 

-- Compile octomap using: make
-- Install octomap using: make install
--     (be sure to set the correct CMAKE_INSTALL_PREFIX before)
-- Compile API-documentation using: make docs

-- 

-- octovis building as Release
-- Found octomap version: 1.10.0
-- Could NOT find OpenGL (missing: OPENGL_opengl_LIBRARY OPENGL_glx_LIBRARY OPENGL_INCLUDE_DIR) 
-- QGLViewer includes found in /home/brain-spark/Octomap/octomap/octovis/src/extern/QGLViewer
-- QGLViewer library not found.
-- Trying to build libQGLViewer from source in /home/brain-spark/Octomap/octomap/octovis/src/extern/QGLViewer
-- 	 generating Makefile using qmake
-- 	 building library (this may take some time...)
make: *** No targets specified and no makefile found.  Stop.
CMake Warning at octovis/CMakeModules/FindQGLViewer.cmake:111 (MESSAGE):
  Could not find libQGLViewer.so, failed to build?
Call Stack (most recent call first):
  octovis

### As we can see, the octomap is updated with the new point that is far away from the existing points. This is not possible with a regular voxel grid.

## 3. Adaptive resolution comparison

### Octomap has the ability to adaptively change the resolution of the octree based on the points being added. This is not possible with a regular voxel grid.

In [16]:
voxel_grid = RegularVoxelGrid(resolution=0.25, max_coord=5, min_coord=-5)
# Generate dense points in a small area (high resolution needed)
dense_points = np.random.rand(1000, 3)

# Generate sparse points in a larger area (low resolution needed)
sparse_points = (np.random.rand(10, 3) * 2) + 2.5

# Combine dense and sparse points
points = np.vstack((dense_points, sparse_points))
voxel_grid.add_point_to_voxel_grid(points)

In [17]:
visualize_voxel_grid(voxel_grid.get_voxel_grid_points())

[Open3D WARNING] GLFW Error: X11: Failed to open display :1
[Open3D WARNING] Failed to initialize GLFW
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.


Authorization required, but no authorization protocol specified


In [18]:
voxel_grid.write_voxel_grid_points_to_txt(filename="adaptive_voxel_grid_points.txt")

In [19]:
voxel_grid.save_voxel_grid_ply()
print(f"Saved voxel map representation space : {voxel_grid.get_saved_voxel_map_disk_size()} MB")

Saved voxel map representation space : 0.4395942687988281 MB


In [20]:
!cd octomap/build && cmake .. && make
!./octomap/bin/simple_example
!octovis octree_from_point_cloud.bt

-- 

-- octomap building as Release
-- Could NOT find Doxygen (missing: DOXYGEN_EXECUTABLE) 
-- 

-- Compile octomap using: make
-- Install octomap using: make install
--     (be sure to set the correct CMAKE_INSTALL_PREFIX before)
-- Compile API-documentation using: make docs

-- 

-- octovis building as Release
-- Found octomap version: 1.10.0
-- Could NOT find OpenGL (missing: OPENGL_opengl_LIBRARY OPENGL_glx_LIBRARY OPENGL_INCLUDE_DIR) 
-- QGLViewer includes found in /home/brain-spark/Octomap/octomap/octovis/src/extern/QGLViewer
-- QGLViewer library not found.
-- Trying to build libQGLViewer from source in /home/brain-spark/Octomap/octomap/octovis/src/extern/QGLViewer
-- 	 generating Makefile using qmake
-- 	 building library (this may take some time...)
make: *** No targets specified and no makefile found.  Stop.
CMake Warning at octovis/CMakeModules/FindQGLViewer.cmake:111 (MESSAGE):
  Could not find libQGLViewer.so, failed to build?
Call Stack (most recent call first):
  octovis

In [21]:
print(f"Saved octomap space: {os.path.getsize('octree_from_point_cloud.bt') / (1024 * 1024)} MB")

Saved octomap space: 0.06645870208740234 MB


## Ray Casting

### Ray casting is a method to mark free spaces along a ray from the sensor origin to the end of the ray. This is useful for collision detection and path planning. Octomap provides a ray casting method to check if a ray is free from obstacles. This is not possible with a regular voxel grid. During octomapping, when we insert ray casting between sensor origin and the detected endpoint from the lidar, we can mark free spaces in the octomap. 

### Ray casting can also be used to see if a ray starting from a point, in a direction intersects with some obstacle or not within a max range. This is useful for path planning and collision detection.

### Modify simple_example.cpp to do ray casting during octomap to insert free cells during adding points from pointcloud. Then in octovis press F to view the free space pop up. 

In [22]:
!cp simple_example.cpp octomap/octomap/src
!cd octomap/build && cmake .. && make
!./octomap/bin/simple_example
!octovis octree_from_point_cloud.bt

-- 

-- octomap building as Release
-- Could NOT find Doxygen (missing: DOXYGEN_EXECUTABLE) 
-- 

-- Compile octomap using: make
-- Install octomap using: make install
--     (be sure to set the correct CMAKE_INSTALL_PREFIX before)
-- Compile API-documentation using: make docs

-- 

-- octovis building as Release
-- Found octomap version: 1.10.0
-- Could NOT find OpenGL (missing: OPENGL_opengl_LIBRARY OPENGL_glx_LIBRARY OPENGL_INCLUDE_DIR) 
-- QGLViewer includes found in /home/brain-spark/Octomap/octomap/octovis/src/extern/QGLViewer
-- QGLViewer library not found.
-- Trying to build libQGLViewer from source in /home/brain-spark/Octomap/octomap/octovis/src/extern/QGLViewer
-- 	 generating Makefile using qmake
-- 	 building library (this may take some time...)
make: *** No targets specified and no makefile found.  Stop.
CMake Warning at octovis/CMakeModules/FindQGLViewer.cmake:111 (MESSAGE):
  Could not find libQGLViewer.so, failed to build?
Call Stack (most recent call first):
  octovis

### Add code to do ray casting for collision detection along a specific direction in simple_example.cpp